In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8-notebook")
import seaborn as sns

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm

from data.cleaning import get_dataset_from_pickle
from utils import adfuller_test, create_weekly_heatmap

import itertools

# City of Los Angeles: Proposing a Strategy for Optimizing Parking Enforcement Deployment

**Author: Evan Gabrielson**

---

A strategy for improving the efficiency of LADOT's Parking Enforcement division should function to save the city and citizenry of Los Angeles money. From public data published by the City of LA on their [Open Budget Explorer](https://openbudget.lacity.org/#!/year/2023/operating/0/department_name/Transportation/0/program_name/Parking+Enforcement+Services/0/source_fund_name), we can see that full and part-time salaries as well as overtime account for over 99% of allocated funding. While the official number of officers is not published publicly, we can use budget data and Indeed job postings by the City of LA to create a heuristic for the number of Full-Time Officers (FTO) working for the Parking Enforcement division.

$$\text{FTO Salary} = (\$23.00 \text{ per hour}) * (40 \text{ hours per week}) * (52 \text{ weeks per year}) =  \$47,840 \text{ annually}$$
$$\text{Number of FTOs} = \$58,311,479 \text{ budgeted for salaries } / \text{ } \$47,840 \text{ per FTO salary} = 1,218 \text{ FTOs}$$

In [ ]:
fto_salary = 23 * 40 * 52
num_ftos = 58.3e6 // fto_salary
print(f"Number of FTOs: {num_ftos}")


It appears that over **1218 full-time officer units** are working for the LADOT Parking Enforcement division. 

As previously stated, to optimize the City of LA's budget, profits must be maximized per officer deployed in a given region for a given shift. We can use historical parking citation data to understand the distribution of citations (1) spatially, across the various regions of Los Angeles, and (2) temporally, across the different days of the week. The citation density distribution is significant to the City of LA's budget optimization problem because it is __directly proportional to the maximum potential revenue that can be generated by the parking enforcement department__. Once a predictor for this distribution is known, we can better estimate the demand for enforcement officers by region and time to maximize revenue per officer.

In [2]:
# Load timeseries
citations_ts = get_dataset_from_pickle('data/pickle/timeseries.pickle')
citations_ts.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 437770 entries, 2013-12-31 07:00:00 to 2023-12-27 10:00:00
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   district        437770 non-null  category
 1   citation_count  437770 non-null  int64   
dtypes: category(1), int64(1)
memory usage: 7.1 MB


For the sake of the simplicity of this initial study, we will focus on citation data only in the district of Hollywood. We will revisit the other districts in a final modeling stage to ensure district-to-district discrepancies are correctly incorporated in the models and to allow a complete view of the City of LA's potential budget savings.

In [3]:
# Extract Hollywood-specific data
hollywood_ts = citations_ts[citations_ts['district'] == 'Hollywood']['citation_count']
hollywood_ts.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 87555 entries, 2013-12-31 08:00:00 to 2023-12-27 10:00:00
Series name: citation_count
Non-Null Count  Dtype
--------------  -----
87555 non-null  int64
dtypes: int64(1)
memory usage: 1.3 MB


#### Looking at Hollywood's Mean Weekly Citation Density

The most rudimentary approach to predicting citation density by hour and weekday is to look at an average of the citation density values across the entire timeseries for each combination of hour and weekday. It would be reasonable to assume that even this basic approach could represent a deployment scheme offering already considerable savings for the City of Los Angeles. Later on, we will examine models that take into account week-by-week differences in citation density to further enhance our savings and deployment demand prediction service.

In [ ]:
create_weekly_heatmap(hollywood_ts)

#### Check for stationarity

Before timeseries modeling can begin, we'll use the Dickey-Fuller test for stationarity and isolate districts that are not stationary on a daily basis.

**Aside:** Testing for non-stationarity with Dickey Fuller

It is important for the sake of modeling and regression techniques that our timeseries is stationary. The Dickey Fuller test allows us to determine whether our timeseries is stationary by trying to establish a value for coefficient $\rho$ in the following equation assuming our timeseries is non-stationary:

$$
x_t = \alpha + \rho x_{t-1} + \epsilon_t \quad \text{where } \alpha = \text{drift \& } \epsilon_t = \text{error term}
$$

By the definition of stationarity, we want to determine if the change in our timeseries ($\Delta x_t$) has a tendency to return to some mean. This would only occur if $\rho$ was less than 1.

$$
\Delta x_t = (\rho - 1) x_{t-1} + \epsilon_t
$$

$$
H_0 : \rho = 1 \quad \text{stationary}\\
H_1 : \rho < 1 \quad \text{non stationary}
$$

In [ ]:
# Print (Augmented) Dickey-Fuller test results
adfuller_test(hollywood_ts)

### Basic Model - ARIMA

Let's split the training and test set and get a baseline model that predicts a constant citation count for all future timesteps. We'll use an ARIMA model with 0 autoregressive (AR) terms and 0 moving average (MA) filter terms to find the future citation count.

In [ ]:
TRAIN_SPLIT = 0.8
split_index = int(hollywood_ts.shape[0] * TRAIN_SPLIT)
train_ts, test_ts = hollywood_ts.iloc[:split_index], hollywood_ts.iloc[split_index:]

In [ ]:
X_train, y_train = train_ts.index, train_ts.values
X_test, y_test = test_ts.index, test_ts.values

In [ ]:
arima = ARIMA(train_ts, order=(0, 0, 0), enforce_invertibility=False, enforce_stationarity=False)
results = arima.fit()
results.summary()

The [Akaike Information Criterion (AIC)](https://iowabiostat.github.io/research-highlights/joe/Cavanaugh_Neath_2019.pdf) value is an estimator of model quality, which when minimized results in the optimal model in terms of simplicity and accuracy. We can compare the AIC value of our Basic ARIMA Model (in this case: 902324) with the AIC value of more advanced modeling techniques to determine whether or not our more advanced models should be preferred.

In [ ]:
results.plot_diagnostics(figsize=(10,10))

In [ ]:
# Plot the trivial model

In [ ]:
# Print RSME and ACF plots

#### SARIMA Model
Capturing seasonality is important, we'll use a 24 hour period to try and pick up any daily seasonality. In order to isolate the best parameters, we'll use the `auto_arima` function to search across autoregressive (AR), differencing (I) and moving average (MA) terms, as well as seasonal components.

In [ ]:
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

# Define the p, d, q parameters to take any value between 0 and 2
p = d = q = range(0, 2)

# Generate all different combinations of p, d, and q
pdq = list(itertools.product(p, d, q))

# Basis represents number of periods in a "season" (7*24 = 168 weekly data points)
basis = [7*24]
seasonal_pdq = list(itertools.product(p, d, q, basis))

# List to store results
results = []

# Grid search for SARIMAX parameters
for order in pdq:
    for seasonal_order in seasonal_pdq:
        try:
            # SARIMAX Model
            model = sm.tsa.SARIMAX(hollywood_ts, order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
            # Fit model
            model_fit = model.fit(disp=False, maxiter=200, method='lbfgs')
            
            # Store results in a dictionary
            results.append({
                'p': order[0],
                'd': order[1],
                'q': order[2],
                'P': seasonal_order[0],
                'D': seasonal_order[1],
                'Q': seasonal_order[2],
                's': basis,
                'AIC': model_fit.aic
            })
            
            print(f'SARIMAX{order} - AIC: {model_fit.aic}')
            
        except Exception as e:
            print(f'SARIMAX{order} failed with error: {e}')
            continue

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Sort results by AIC value
results_df = results_df.sort_values(by='AIC', ascending=True).reset_index(drop=True)

# Display the results
print(results_df)


In [ ]:
best_sarima = SARIMAX(train_ts, order=(), seasonal_order=(), enforce_invertibility=False, enforce_stationarity=False)
results = best_sarima.fit()
results.summary()

In [ ]:
results.plot_diagnostics(figsize=(10,10))

In [ ]:
# Plot SARIMA predictions